# المرحلة 1: الاستيرادات والمسارات

In [1]:
import os
import gc
import time
import pandas as pd
import numpy as np



# مسار الداتا على Kaggle

In [2]:
INPUT_DIR = "/kaggle/input/ecommerce-behavior-data-from-multi-category-store"
CANDIDATES = ["2019-Oct.csv.gz", "2019-Oct.csv"]
INPUT_PATH = None
for name in CANDIDATES:
    p = os.path.join(INPUT_DIR, name)
    if os.path.exists(p):
        INPUT_PATH = p
        break
print("Using:", INPUT_PATH)



Using: /kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv


# مجلد الإخراج


In [3]:
OUTPUT_PATH = "/kaggle/working/oct_daily_eventtype_summary.csv"


# المرحلة 2: إعدادات القراءة
# نقتصر على الأعمدة المطلوبة للتلخيص

In [4]:

USECOLS = ["event_time", "event_type", "price", "user_session"]




# أنواع بيانات لتقليل الذاكرة


In [5]:
DTYPES = {
    "event_type": "category",
    "price": "float32",
    "user_session": "category",
}
PARSE_DATES = ["event_time"]



# حجم الدفعات: يمكن تعديله لاحقاً


In [6]:
CHUNK_SIZE = 1_000_000  # جرّب 500_000 إذا ظهرت مشكلة ذاكرة

print("Config ready:", {"usecols": USECOLS, "chunk_size": CHUNK_SIZE})

Config ready: {'usecols': ['event_time', 'event_type', 'price', 'user_session'], 'chunk_size': 1000000}


# المرحلة 3: قراءة عينة صغيرة للفحص


In [7]:
sample = pd.read_csv(
    INPUT_PATH,
    usecols=USECOLS,
    dtype=DTYPES,
    parse_dates=PARSE_DATES,
    nrows=200_000
)
print(sample.dtypes)
print(sample.head(3))
print("Rows:", len(sample))


event_time      datetime64[ns, UTC]
event_type                 category
price                       float32
user_session               category
dtype: object
                 event_time event_type       price  \
0 2019-10-01 00:00:00+00:00       view   35.790001   
1 2019-10-01 00:00:00+00:00       view   33.200001   
2 2019-10-01 00:00:01+00:00       view  543.099976   

                           user_session  
0  72d76fde-8bb3-4e00-8c23-a032dfed738c  
1  9333dfbd-b87a-4708-9857-6336556b0fcc  
2  566511c2-e2e3-422b-b695-cf8e6e792ca8  
Rows: 200000


# المرحلة 4: تنفيذ chunking والتجميع المرحلي


In [8]:
aggregates = []
start = time.time()

reader = pd.read_csv(
    INPUT_PATH,
    usecols=USECOLS,
    dtype=DTYPES,
    parse_dates=PARSE_DATES,
    chunksize=CHUNK_SIZE
)

for i, chunk in enumerate(reader, start=1):
    # تنظيف بسيط
    chunk = chunk.dropna(subset=["event_time", "event_type"])
    # اليوم (UTC)
    chunk["day"] = chunk["event_time"].dt.floor("D")
    # سعر خالٍ من NaN
    chunk["price"] = chunk["price"].fillna(0.0).astype("float32")

    # تجميع مرحلي
    agg = (
        chunk.groupby(["day", "event_type"])
             .agg(
                 events=("event_type", "size"),
                 total_price=("price", "sum"),
                 unique_sessions=("user_session", "nunique")
             )
             .reset_index()
    )
    aggregates.append(agg)

    # تحرير ذاكرة
    del chunk, agg
    gc.collect()

    if i % 5 == 0:
        print(f"Processed ~{i*CHUNK_SIZE:,} rows")

print(f"Finished chunking in {(time.time()-start)/60:.2f} min")


/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk.groupby(["day", "event_type"])
/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk.groupby(["day", "event_type"])
/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk.groupby(["day", "event_type"])
/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of obser

Processed ~5,000,000 rows


/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk.groupby(["day", "event_type"])
/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk.groupby(["day", "event_type"])
/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk.groupby(["day", "event_type"])
/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of obser

Processed ~10,000,000 rows


/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk.groupby(["day", "event_type"])
/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk.groupby(["day", "event_type"])
/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk.groupby(["day", "event_type"])
/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of obser

...

/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk.groupby(["day", "event_type"])
/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk.groupby(["day", "event_type"])
/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk.groupby(["day", "event_type"])
/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of obser

Processed ~40,000,000 rows


/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk.groupby(["day", "event_type"])
/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk.groupby(["day", "event_type"])


Finished chunking in 8.26 min


/tmp/ipykernel_37/229769290.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk.groupby(["day", "event_type"])


# المرحلة 5: دمج النتائج من الدُفعات وحفظ الملخص
# دمج قوائم التجميع

In [9]:

summary = pd.concat(aggregates, ignore_index=True)

# إعادة تجميع نهائي بعد الدمج لتوحيد القيم عبر الدُفعات
summary = (
    summary.groupby(["day", "event_type"], observed=True, as_index=False)
           .agg(
               events=("events", "sum"),
               total_price=("total_price", "sum"),
               unique_sessions=("unique_sessions", "sum")
           )
           .sort_values(["day", "event_type"])
)

# حفظ الناتج
summary.to_csv(OUTPUT_PATH, index=False)

print("Saved:", OUTPUT_PATH)
display(summary.head(10))


Saved: /kaggle/working/oct_daily_eventtype_summary.csv


,day,event_type,events,total_price,unique_sessions
0,2019-10-01 00:00:00+00:00,cart,16658,6123619.5,10175
1,2019-10-01 00:00:00+00:00,purchase,19307,6275964.0,16260
2,2019-10-01 00:00:00+00:00,view,1208280,358235648.0,270392
3,2019-10-02 00:00:00+00:00,cart,17268,6272537.0,10572
4,2019-10-02 00:00:00+00:00,purchase,19469,6213628.5,16429
5,2019-10-02 00:00:00+00:00,view,1154591,345087360.0,262785
6,2019-10-03 00:00:00+00:00,cart,19323,7076894.5,11544
7,2019-10-03 00:00:00+00:00,purchase,19255,6233783.0,16170
8,2019-10-03 00:00:00+00:00,view,1088725,326170624.0,242360
9,2019-10-04 00:00:00+00:00,cart,43829,15607456.0,26726


# المرحلة 6: حفظ نسخة مضغوطة وإضافات اختيارية
# إضافة متوسط السعر لكل حدث

In [10]:

summary["avg_price_per_event"] = (summary["total_price"] / summary["events"]).astype("float32")

# حفظ نسخة CSV عادية ومضغوطة
csv_path = "/kaggle/working/oct_daily_eventtype_summary.csv"
csv_gz_path = "/kaggle/working/oct_daily_eventtype_summary.csv.gz"

summary.to_csv(csv_path, index=False)
summary.to_csv(csv_gz_path, index=False, compression="gzip")

# عرض حجم الملفات
size_csv = os.path.getsize(csv_path) / (1024*1024)
size_gz = os.path.getsize(csv_gz_path) / (1024*1024)
print(f"CSV size: {size_csv:.2f} MB, GZ size: {size_gz:.2f} MB")

display(summary.head(10))


CSV size: 0.01 MB, GZ size: 0.00 MB


,day,event_type,events,total_price,unique_sessions,avg_price_per_event
0,2019-10-01 00:00:00+00:00,cart,16658,6123619.5,10175,367.608337
1,2019-10-01 00:00:00+00:00,purchase,19307,6275964.0,16260,325.061584
2,2019-10-01 00:00:00+00:00,view,1208280,358235648.0,270392,296.483978
3,2019-10-02 00:00:00+00:00,cart,17268,6272537.0,10572,363.246307
4,2019-10-02 00:00:00+00:00,purchase,19469,6213628.5,16429,319.154999
5,2019-10-02 00:00:00+00:00,view,1154591,345087360.0,262785,298.882782
6,2019-10-03 00:00:00+00:00,cart,19323,7076894.5,11544,366.242004
7,2019-10-03 00:00:00+00:00,purchase,19255,6233783.0,16170,323.748779
8,2019-10-03 00:00:00+00:00,view,1088725,326170624.0,242360,299.589539
9,2019-10-04 00:00:00+00:00,cart,43829,15607456.0,26726,356.098846


# عدد الأيام × عدد الأنواع يجب أن يساوي عدد صفوف الملخص تقريباً (عادة 31 × 3 = 93 إن وُجدت ثلاثة أنواع)


In [11]:
print("Rows in summary:", len(summary))
print("Unique days:", summary["day"].nunique())
print("Event types:", summary["event_type"].unique().tolist())

# لا توجد قيم سالبة أو NaN في المقاييس
print("Any NaN?", summary[["events","total_price","unique_sessions","avg_price_per_event"]].isna().any().to_dict())
print("Any negatives?", {
    "events": (summary["events"] < 0).any(),
    "total_price": (summary["total_price"] < 0).any(),
    "unique_sessions": (summary["unique_sessions"] < 0).any(),
})


Rows in summary: 93
Unique days: 31
Event types: ['cart', 'purchase', 'view']
Any NaN? {'events': False, 'total_price': False, 'unique_sessions': False, 'avg_price_per_event': False}
Any negatives? {'events': False, 'total_price': False, 'unique_sessions': False}


# ضغط مجلد العمل كأرشيف واحد اختياري


In [12]:
import os
import zipfile

archive_path = "/kaggle/working/oct_summary_artifacts.zip"
with zipfile.ZipFile(archive_path, mode="w", compression=zipfile.ZIP_DEFLATED) as z:
    z.write("/kaggle/working/oct_daily_eventtype_summary.csv", arcname="oct_daily_eventtype_summary.csv")
    z.write("/kaggle/working/oct_daily_eventtype_summary.csv.gz", arcname="oct_daily_eventtype_summary.csv.gz")
print("Archive ready:", archive_path)


Archive ready: /kaggle/working/oct_summary_artifacts.zip


# المرحلة 9: مؤشرات إضافية يومية
# تحويل الملخص إلى شكل محوري للحصول على عدد الأحداث لكل نوع

In [13]:

pivot = summary.pivot(index="day", columns="event_type", values="events").fillna(0).reset_index()

# حساب معدل التحويل اليومي
pivot["purchase_rate"] = (pivot.get("purchase", 0) / pivot.get("view", 1)).astype("float32")

# متوسط السعر لكل جلسة يومياً: مجموع السعر لكل يوم / مجموع الجلسات لكل يوم
sessions_daily = summary.groupby("day", as_index=False)["unique_sessions"].sum().rename(columns={"unique_sessions":"sessions_total"})
price_daily = summary.groupby("day", as_index=False)["total_price"].sum().rename(columns={"total_price":"total_price_total"})

metrics = pivot.merge(sessions_daily, on="day").merge(price_daily, on="day")
metrics["avg_price_per_session"] = (metrics["total_price_total"] / metrics["sessions_total"].clip(lower=1)).astype("float32")

out_metrics = "/kaggle/working/oct_daily_metrics.csv"
metrics.to_csv(out_metrics, index=False)
print("Saved:", out_metrics)
metrics.head()


Saved: /kaggle/working/oct_daily_metrics.csv


,day,cart,purchase,view,purchase_rate,sessions_total,total_price_total,avg_price_per_session
0,2019-10-01 00:00:00+00:00,16658,19307,1208280,0.015979,296827,370635232.0,1248.657349
1,2019-10-02 00:00:00+00:00,17268,19469,1154591,0.016862,289786,357573536.0,1233.922729
2,2019-10-03 00:00:00+00:00,19323,19255,1088725,0.017686,270074,339481312.0,1256.993652
3,2019-10-04 00:00:00+00:00,43829,27041,1346320,0.020085,351922,423692832.0,1203.939575
4,2019-10-05 00:00:00+00:00,35497,23494,1271348,0.018480,319885,395676480.0,1236.933472


# المرحلة 10: تحسينات سريعة
# مثال groupby مع observed=True لإسكات التحذير مستقبلاً
# ملاحظة: تم تطبيق observed=True بالفعل في التجميع النهائي

In [14]:


# تحقق جودة أساسي
assert summary["events"].ge(0).all(), "Negative events detected"
assert summary["total_price"].ge(0).all(), "Negative total_price detected"
assert summary["unique_sessions"].ge(0).all(), "Negative unique_sessions detected"

# توصية اختيارية: تقليل dtype للحجم
summary_optimized = summary.copy()
summary_optimized["events"] = summary_optimized["events"].astype("int32")
summary_optimized["unique_sessions"] = summary_optimized["unique_sessions"].astype("int32")
summary_optimized["total_price"] = summary_optimized["total_price"].astype("float32")

opt_path = "/kaggle/working/oct_daily_eventtype_summary_optimized.csv.gz"
summary_optimized.to_csv(opt_path, index=False, compression="gzip")
print("Saved optimized:", opt_path)


Saved optimized: /kaggle/working/oct_daily_eventtype_summary_optimized.csv.gz


# المرحلة 11: تعميم على أشهر متعددة ودمج النتائج


In [15]:
INPUT_DIR = "/kaggle/input/ecommerce-behavior-data-from-multi-category-store"
files = ["2019-Oct.csv", "2019-Nov.csv", "2019-Oct.csv.gz", "2019-Nov.csv.gz"]

def find_existing(paths):
    for p in paths:
        full = os.path.join(INPUT_DIR, p)
        if os.path.exists(full):
            yield full

targets = list(find_existing(files))
print("Found files:", targets)

def process_file(path, chunk_size=1_000_000):
    aggs = []
    for chunk in pd.read_csv(
        path,
        usecols=["event_time","event_type","price","user_session"],
        dtype={"event_type":"category","price":"float32","user_session":"category"},
        parse_dates=["event_time"],
        chunksize=chunk_size
    ):
        chunk = chunk.dropna(subset=["event_time","event_type"])
        chunk["day"] = chunk["event_time"].dt.floor("D")
        chunk["price"] = chunk["price"].fillna(0.0).astype("float32")
        agg = (
            chunk.groupby(["day","event_type"], observed=True)
                 .agg(events=("event_type","size"),
                      total_price=("price","sum"),
                      unique_sessions=("user_session","nunique"))
                 .reset_index()
        )
        aggs.append(agg)
        del chunk, agg
        gc.collect()
    df = pd.concat(aggs, ignore_index=True)
    df = df.groupby(["day","event_type"], observed=True, as_index=False).agg(
        events=("events","sum"),
        total_price=("total_price","sum"),
        unique_sessions=("unique_sessions","sum")
    )
    df["month"] = df["day"].dt.to_period("M").astype(str)
    return df

multi = []
for p in targets:
    print("Processing:", p)
    multi.append(process_file(p, chunk_size=1_000_000))

if multi:
    all_months = pd.concat(multi, ignore_index=True).sort_values(["day","event_type"])
    all_path = "/kaggle/working/multi_month_eventtype_summary.csv.gz"
    all_months.to_csv(all_path, index=False, compression="gzip")
    print("Saved:", all_path)
    all_months.head()
else:
    print("No additional monthly files found; skipping multi-month aggregation.")


Found files: ['/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv', '/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv']
Processing: /kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv


/tmp/ipykernel_37/1315790664.py:41: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["month"] = df["day"].dt.to_period("M").astype(str)


Processing: /kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv
Saved: /kaggle/working/multi_month_eventtype_summary.csv.gz


/tmp/ipykernel_37/1315790664.py:41: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["month"] = df["day"].dt.to_period("M").astype(str)


# المرحلة 12: مؤشرات شهرية عبر عدة أشهر
# إن كان all_months موجوداً من المرحلة السابقة

In [16]:

try:
    df = all_months.copy()
    pivot_m = df.pivot_table(index="month", columns="event_type", values="events", aggfunc="sum", fill_value=0, observed=True)
    pivot_m = pivot_m.reset_index()
    pivot_m["purchase_rate"] = (pivot_m.get("purchase", 0) / pivot_m.get("view", 1)).astype("float32")

    # إجمالي السعر والجلسات شهرياً
    price_m = df.groupby("month", as_index=False)["total_price"].sum().rename(columns={"total_price":"total_price_total"})
    sess_m = df.groupby("month", as_index=False)["unique_sessions"].sum().rename(columns={"unique_sessions":"sessions_total"})

    monthly = pivot_m.merge(price_m, on="month").merge(sess_m, on="month")
    monthly["avg_price_per_session"] = (monthly["total_price_total"] / monthly["sessions_total"].clip(lower=1)).astype("float32")

    monthly_path = "/kaggle/working/monthly_metrics.csv"
    monthly.to_csv(monthly_path, index=False)
    print("Saved:", monthly_path)
    monthly.head()
except NameError:
    print("Skip: multi-month dataframe not found; run stage 11 first if needed.")


Saved: /kaggle/working/monthly_metrics.csv


# المرحلة 13: تحديث الأرشيف بملفات المؤشرات


In [17]:
import zipfile

extra = [
    "/kaggle/working/oct_daily_metrics.csv",
    "/kaggle/working/oct_daily_eventtype_summary_optimized.csv.gz",
    "/kaggle/working/multi_month_eventtype_summary.csv.gz",
    "/kaggle/working/monthly_metrics.csv",
]
archive_path = "/kaggle/working/oct_summary_artifacts_extended.zip"
with zipfile.ZipFile(archive_path, mode="w", compression=zipfile.ZIP_DEFLATED) as z:
    for f in extra:
        if os.path.exists(f):
            z.write(f, arcname=os.path.basename(f))
print("Archive ready:", archive_path)


Archive ready: /kaggle/working/oct_summary_artifacts_extended.zip
